In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Food inspection rating data

In [2]:
df = pd.read_csv("Food_Safety_Inspections___Current_Ratings.csv")
df.head()

,County,Inspection Grade,Inspection Date,Establishment Types,Owner Name,Trade Name,Street,City,State Code,Zip Code,Deficiency Number,Deficiency Description,Georeference
0,Albany,A,04/18/2022,W,100 ACRE WOOD VINEYARD LLC,CLOVERPOND VINEYARD,100 MARIPOSA LANE,ALTAMONT,NY,12009,NaN,NaN,POINT (-73.96214 42.72462)
1,Albany,A,02/01/2023,AC,1095 LOUDON INC,BOGHT CORNERS MOBIL,1095 LOUDON ROAD,COHOES,NY,12047,NaN,NaN,POINT (-73.74413 42.78365)
2,Albany,A,05/07/2021,AC,1095 LOUDON INC,BOGHT CORNERS MOBIL,1095 LOUDON ROAD,COHOES,NY,12047,NaN,NaN,POINT (-73.74413 42.78365)
3,Albany,C,01/03/2023,AC,1250 SELKIRK INC,1250 SELKIRK,1250 RTE 9W & 396,SELKIRK,NY,12158,12A,- Several deli meats stored in display cooler ...,POINT (-73.8073 42.54758)
4,Albany,C,01/03/2023,AC,1250 SELKIRK INC,1250 SELKIRK,1250 RTE 9W & 396,SELKIRK,NY,12158,09F,- Steel wool ball above equipment wash sink in...,POINT (-73.8073 42.54758)


In [3]:
df.columns

Index(['County', 'Inspection Grade', 'Inspection Date', 'Establishment Types',
       'Owner Name', 'Trade Name', 'Street', 'City', 'State Code', 'Zip Code',
       'Deficiency Number', 'Deficiency Description', 'Georeference'],
      dtype='object')

In [4]:
df=df.drop(['Deficiency Number', 'Deficiency Description'],axis=1)

In [5]:
df.shape

(183481, 11)

In [6]:
df.isna().sum()

County                    0
Inspection Grade          0
Inspection Date           0
Establishment Types       0
Owner Name                0
Trade Name               55
Street                    0
City                     18
State Code                0
Zip Code                  0
Georeference           1293
dtype: int64

In [7]:
df['Inspection Grade'].value_counts()

C    103127
B     40279
A     40075
Name: Inspection Grade, dtype: int64

In [8]:
df=df.dropna()

In [9]:
df=df.reset_index(drop=True)

In [10]:
df.nunique()

County                    62
Inspection Grade           3
Inspection Date          520
Establishment Types      159
Owner Name             22467
Trade Name             26715
Street                 28153
City                    1372
State Code                 1
Zip Code                1489
Georeference           26661
dtype: int64

In [11]:
lat=[]
lon=[]
prob_ind=[]
print(type(df['Georeference'][4]))
for i in range(len(df)):
    if type(df['Georeference'][i])==str:
        coor = df['Georeference'][i].split("(")[1].split(")")[0].split()
        coor = list(map(float, coor))
        lon.append(coor[0])
        lat.append(coor[1])
    else:
        prob_ind.append(i)

<class 'str'>


In [12]:
df = df.drop(prob_ind)
df = df.reset_index(drop=True)

In [13]:
df['longitude']=lon
df['latitude']=lat

df = df.loc[(df['longitude'] > (-90)) & (df['longitude'] < (-73)) & (df['latitude'] > 40)]
df = df.reset_index(drop=True)

In [14]:
# which shows that out of total food businesses 38967 are of A grade. which means that 78% of those are of B or C ratings

# (df['Inspection Grade'].value_counts()[3] + df['Inspection Grade'].value_counts()[2])/len(df)
df['Inspection Grade'].value_counts()

C    102031
B     39683
A     38967
Name: Inspection Grade, dtype: int64

### Selecting the Food Establishments

In [15]:
df_new = df.loc[(df['Establishment Types'] == "ACD") | (df['Establishment Types'] == "CDK") | (df['Establishment Types'] == "ACK")
                    | (df['Establishment Types'] == "ABCH") | (df['Establishment Types'] == "ABC")]

In [16]:
df_new = df_new.reset_index(drop=True)

In [17]:
df_new.shape

(12808, 13)

In [18]:
df_new_un = df_new.drop_duplicates()

In [19]:
df_new_un = df_new_un.reset_index(drop=True)

In [20]:
df_new_un_name = df_new_un.drop_duplicates(subset="Trade Name",keep="last")

In [21]:
df_new_un_name = df_new_un_name.reset_index(drop=True)

In [22]:
df_new_un_name.shape

(1465, 13)

### tourist dataset

In [23]:
tour = pd.read_csv("tourist places nys.csv")

In [24]:
tour.head()

,Place,latitude,longitude
0,Statue of Liberty,40.6892,-74.0445
1,Ellis Island,40.6995,-74.0396
2,Times Square,40.7580,-73.9855
3,Finger Lakes,42.7238,-76.9297
4,Lake Placid,44.2795,-73.9799


In [25]:
tour.shape

(15, 3)

### Universities and Colleges data

In [26]:
more_unis = pd.read_csv("updated more unis.csv")
more_unis.head()

,Unnamed: 0,School,City,Enroll,In-State Cost,Out-State Cost,latitude,longitude
0,0,ASA College,Brooklyn,"2,965","$33,085","$33,085",40.691526,-73.985998
1,1,Adelphi University,Garden City,"7,584","$63,605","$63,605",40.719764,-73.651972
2,2,Albany College of Pharmacy and Health Sciences,Albany,"1,118","$51,555","$51,555",42.652026,-73.778614
3,3,Albany Law School,Albany,566,$0,$0,42.651546,-73.777462
4,4,Albany Medical College,Albany,835,$0,$0,42.654123,-73.777345


In [27]:
more_unis.describe()

,Unnamed: 0,latitude,longitude
count,196.00000,196.000000,196.000000
mean,97.50000,41.889517,-75.637860
std,56.72448,1.567033,4.259472
min,0.00000,29.443141,-119.087127
25%,48.75000,40.797599,-76.504700
50%,97.50000,42.106931,-74.009522
75%,146.25000,42.914166,-73.892884
max,195.00000,46.862419,-71.304583


In [28]:
more_unis = more_unis.loc[(more_unis['longitude'] > (-79)) & (more_unis['longitude'] < (-73)) & (more_unis['latitude'] > 40) & (more_unis['latitude'] < 45)]
more_unis = more_unis.reset_index(drop=True)

In [29]:
more_unis.shape

(186, 8)

# Visuals

### Food establishments types and their ratings

In [34]:
url="https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json"
source = alt.topo_feature(url,feature="cb_2015_new_york_county_20m")
# Read the TopoJSON file as a GeoDataFrame
ny_counties = gpd.read_file('ny_county.json', driver='TopoJSON')

# Compute the centroids of each county
ny_counties['clong'] = ny_counties.centroid.x
ny_counties['clat'] = ny_counties.centroid.y
ny_counties = ny_counties.drop(['geometry'],axis=1)

# based on clicking the bars of the countlplot we can display the change in the map
click = alt.selection_single(encodings=['color'])

est_col_scale = {"ABC":"yellow","ABCH":"black","ACD":"orange","ACK":"cyan","CDK":"purple","A":"green","B":"red","C":"blue"}

counts = alt.Chart(df_new_un_name).mark_bar().encode(
    y=alt.Y("Establishment Types:N",title=None),
    color=alt.condition(click, alt.Color('Establishment Types:N', scale=alt.Scale(domain=list(est_col_scale.keys()), range=list(est_col_scale.values()))), alt.value('lightgrey'), legend=None),
).properties(
    width=250,
    height=120,
    title="Establishment types"
).add_selection(click)

# base map
base = alt.Chart(source).mark_geoshape(
    fill="lightgrey",
    stroke="white",
    strokeWidth=1.5
).encode(
    tooltip='properties.NAME:N'
).properties(
        title='Locations of the selected food establishments across New York State',
        width=850,
        height=950).project(type="mercator")

# county names
labels = alt.Chart(ny_counties).mark_text().encode(
    longitude='clong:Q',
    latitude='clat:Q',
    text='NAME:N',
    size=alt.value(7.5),
    opacity=alt.value(0.6)
)

# Points representing food est.
points = alt.Chart(df_new_un_name).mark_circle(size=60).encode(
    longitude="longitude",
    latitude="latitude",
    color=alt.Color('Establishment Types:N',scale=alt.Scale(domain=list(est_col_scale.keys()), range=list(est_col_scale.values())), legend=None),
    tooltip=["County","City","Inspection Grade","Establishment Types","Trade Name"]
).project("mercator").transform_filter(click)


# Filter the data based on the selection
filtered_points = points.add_selection(click)

# Pie chart distribution
pie = alt.Chart(df_new_un_name).mark_arc().encode(
    theta="count(Inspection Grade)",
    color=alt.Color('Inspection Grade:N',scale=alt.Scale(domain=list(est_col_scale.keys()), range=list(est_col_scale.values()))),
    tooltip=["count(Inspection Grade)","Inspection Grade"],
).properties(
    width=250,
    height=250,
    title="Distribution of Inspection Grades"
).project("mercator").transform_filter(click)

# Bar chart distribution
bar = alt.Chart(df_new_un_name).mark_bar().encode(
    x=alt.X("Establishment Types"),
    y=alt.Y("count(Inspection Grade)",stack="normalize"),
    color=alt.Color('Inspection Grade:N',scale=alt.Scale(domain=list(est_col_scale.keys()), range=list(est_col_scale.values()))),
    tooltip=["Inspection Grade","count(Inspection Grade)"]
).properties(
    width=250,
    height=250,
    title="Corresponding Bar chart representation"
).transform_filter(click)


# Combine all of them into a layer chart
visual = (pie | counts | bar) & (base + points + labels)
visual = visual.configure_legend(
    disable=True
).properties(
    title={'text':'Food establishments types and their ratings',
               'subtitle':['Each food business can have multiple types of establishment. "A" : Store, "B" : Bakery, "C" : Food manufacturer',
                          '"D" : Food warehouse, "H" : Wholesale manufacturer, "K" : Vehicle',
                           'Select any of the establishment type to see the distribution','Double click to select all',
                           '\n','\n'],
            'fontSize' : 20,
          'fontWeight':900}
).configure_title(
    anchor='middle'
)
visual
# To save the visual uncomment the below line and run.
# visual.save("Inspection grade distribution.html", embed_options={'actions': False})

alt.VConcatChart(...)

### Food establishments with respect to universities and tourist places

In [35]:
url="https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json"
source = alt.topo_feature(url,feature="cb_2015_new_york_county_20m")
# Read the TopoJSON file as a GeoDataFrame
ny_counties = gpd.read_file('ny_county.json', driver='TopoJSON')

# Compute the centroids of each county
ny_counties['clong'] = ny_counties.centroid.x
ny_counties['clat'] = ny_counties.centroid.y
ny_counties = ny_counties.drop(['geometry'],axis=1)

# Base map
base = alt.Chart(source).mark_geoshape(
    fill="#52b7ee",
    stroke="lightgrey",
    strokeWidth=1.2
).encode(
    tooltip='properties.NAME:N'
).properties(
        title={'text':'Food establishments with respect to universities and tourist places',
               'subtitle':['Yellow squares : Universities and colleges','Dark Green squares : Tourist places',
                           'Dots : Food establishments','Hover over for more details']},
        width=850,
        height=900).project(type="mercator")

# County names
labels = alt.Chart(ny_counties).mark_text().encode(
    longitude='clong:Q',
    latitude='clat:Q',
    text='NAME:N',
    size=alt.value(9),
    opacity=alt.value(0.8)
)

# Tourist places
tours = alt.Chart(tour).mark_square(size=150,color="#4b5320").encode(
    longitude="longitude",
    latitude="latitude",
    tooltip=["Place"],
).project("mercator")


# Food establishments
foods = alt.Chart(df_new_un_name).mark_circle(size=20, color="black").encode(
    longitude="longitude",
    latitude="latitude",
    tooltip=["County","City","Inspection Grade","Establishment Types","Trade Name"],
).project("mercator")

# Universities and colleges
unis = alt.Chart(more_unis).mark_square(size=150,color="yellow").encode(
    longitude="longitude",
    latitude="latitude",
    tooltip=["School"],
).project("mercator")


# Combine all of them into a layer chart
chart = (base + foods + tours + unis + labels)

chart = chart.configure_title(
    fontSize=20,
    offset=-80,
    fontWeight=900
).configure_view(
    strokeWidth=0,
    fill="#fed0b1"
)
chart
# To save the visual, uncomment the below line an run.
# chart.save("Food est, tours and unis.html", embed_options={'actions': False})

alt.LayerChart(...)